In [ ]:
#In this project, you're working for a client who wants to create a model that can predict the price of apartments in the city of Buenos Aires
#  — with a focus on apartments that cost less than $400,000 USD.

import warnings

import matplotlib.pyplot as plt
import pandas as pd
import wqet_grader
from IPython.display import VimeoVideo
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.utils.validation import check_is_fitted

warnings.simplefilter(action="ignore", category=FutureWarning)
wqet_grader.init("Project 2 Assessment")

In [ ]:
#Task 2.1.1: Write a function named wrangle that takes a file path as an argument and returns a DataFrame

def wrangle(file_path):
    df = pd.read_csv(file_path)
    return df

In [ ]:
# Task 2.1.2: Use your wrangle function to create a DataFrame df from the CSV file data/buenos-aires-real-estate-1.csv.

df = wrangle("./data/buenos-aires-real-estate-1.csv")
print("df shape:", df.shape)
df.head()

In [ ]:
# Task 2.1.3: Add to your wrangle function so that the DataFrame it 
# returns only includes apartments in Buenos Aires ("Capital Federal") that cost less than $400,000 USD. 
# Then recreate df from data/buenos-aires-real-estate-1.csv by re-running the cells above. 

def wrangle(file_path):
    df = pd.read_csv(file_path)
    mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
    mask_apt = df["property_type"] == "apartment"
    mask_price = df["price"] < 400000
    df = df[mask_ba & mask_apt & mask_price]
    return df

In [ ]:
# Task 2.1.4: Create a histogram of "surface_covered_in_m2". 
# Make sure that the x-axis has the label "Area [sq meters]" and the plot has the title "Distribution of Apartment Sizes".

plt.hist(df["surface_covered_in_m2"])
plt.xlabel("Area [sq meters]")
plt.title("Distribution of Apartment Sizes")

In [ ]:
# Task 2.1.5: Calculate the summary statistics for df using the describe method.
df.describe()

In [ ]:
# Task 2.1.6: Add to your wrangle function so that it removes observations that are outliers in the `"surface_covered_in_m2"` column. Specifically, all observations should fall between the `0.1` and `0.9` quantiles for `"surface_covered_in_m2"`.

def wrangle(file_path):
    df = pd.read_csv(file_path)
    mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
    mask_apt = df["property_type"] == "apartment"
    mask_price = df["price"] < 400000
    df = df[mask_ba & mask_apt & mask_price]
    
    # Remove outlier `"surface_covered_in_m2"`
    low, high = df["surface_covered_in_m2"].quantile([0.1,0.9])
    mark_area = df["surface_covered_in_m2"].between(low, high)
    
    df = df [mark_area]
    return df

In [ ]:
# Task 2.1.7: Create a scatter plot that shows price ("price_aprox_usd") vs area ("surface_covered_in_m2") in our dataset. Make sure to label your x-axis "Area [sq meters]" and your y-axis "Price [USD]".

